# Tutorial 2: run CANAL with data streams from various batches

In [1]:
import os
os.chdir('/data/wanh/CANAL/') 
import sys
sys.path.append('/data/wanh/CANAL/')
import argparse
from model import *
import time
import resource

## construct the CANAL model

In [2]:
experiments = 'Pancreas'
seed = 1
with open(f'./data/{experiments}/{experiments}_highly_gene_idx.csv') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
      highly_variable_idx = []
      for row in spamreader:
            highly_variable_idx.append(row[0])

highly_variable_idx = [int(i) for i in highly_variable_idx]
highly_variable_idx = np.array(highly_variable_idx)

In [3]:
CANAL = CANAL_model(gpu_option='1')

## stage 1

### load the pre-processed dataset ''Muraro'' 

In [4]:
dataset_stage_1 = "Muraro"
data_path_stage_1 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_1)
adata_stage_1 = sc.read_h5ad(data_path_stage_1)
cell_type_stage_1 = adata_stage_1.obs['cell_type1']
print(adata_stage_1) 
print(np.unique(np.array(cell_type_stage_1), return_counts=True))

AnnData object with n_obs × n_vars = 1598 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['acinar', 'alpha', 'beta', 'delta', 'ductal', 'gamma',
       'mesenchymal'], dtype=object), array([168, 613, 352, 153, 177,  76,  59]))


### fine-tune the CANAL model for the first stage

In [5]:
CANAL.train(experiments = experiments, pre_dataset = "None", dataset = dataset_stage_1, 
            adata = adata_stage_1, cell_type = cell_type_stage_1, current_stage = 1, 
            is_final_stage = False,ckpt_dir = './ckpts/', rehearsal_size = 1000, 
            highly_variable_idx=highly_variable_idx, SEED = seed)

current data: AnnData object with n_obs × n_vars = 1598 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['acinar', 'alpha', 'beta', 'delta', 'ductal', 'gamma',
       'mesenchymal'], dtype=object), array([168, 613, 352, 153, 177,  76,  59]))
model constructing begin!



/data/wanh/CANAL/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


model constructing finished!

label train: [0 1 2 3 4 5 6] 7
label val: [0 1 2 3 4 5 6] 7
  ==  Begin finetuning: | Initial stage | Current stage: 1 | CANAL | Dataset: Pancreas Muraro ==
    ==  Epoch: 1 | Classification Loss: 1.961912 | Representation DL Loss: 0.000000  | Accuracy: 4.5526%  ==
    ==  Epoch: 1 | Validation CLS Loss: 1.950393 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.056088  == | Current accuracy: 0.110390  ==
[[  0   0   8   0   0   0  25]
 [  0   0  21   0   0   0 109]
 [  0   0  31   0   0   0  34]
 [  0   0   9   0   0   0  21]
 [  0   0  22   0   0   0   4]
 [  0   0   5   0   0   0   7]
 [  0   0   9   0   0   0   3]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 1.642274 | Representation DL Loss: 0.000000  | Accuracy: 37.5981%  ==
    ==  Epoch: 2 | Validation CLS Loss: 1.650223 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.162927  == | Current accuracy: 0.392857  ==
[[  0  35   2   0   0   0   0]
 [  0 111   1   0   0 

    ==  Epoch: 17 | Validation CLS Loss: 2.981735 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.936793  == | Current accuracy: 0.964286  ==
[[ 40   0   0   0   0   0   0]
 [  0 128   1   0   0   0   0]
 [  1   1  59   0   1   0   0]
 [  0   0   1  27   0   0   0]
 [  1   0   0   0  28   0   0]
 [  1   2   0   2   0   7   0]
 [  0   0   0   0   0   0   8]]
Patience: 5 / 10
    ==  Epoch: 18 | Classification Loss: 0.522617 | Representation DL Loss: 0.000000  | Accuracy: 98.1162%  ==
    ==  Epoch: 18 | Validation CLS Loss: 3.346835 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.930889  == | Current accuracy: 0.944805  ==
[[ 36   0   0   0   0   0   0]
 [  0 117   0   0   0   0   0]
 [  2   5  49   0   1   1   0]
 [  0   0   0  34   0   0   0]
 [  1   0   1   0  29   0   0]
 [  0   0   0   6   0  14   0]
 [  0   0   0   0   0   0  12]]
Patience: 6 / 10
    ==  Epoch: 19 | Classification Loss: 0.530727 | Representation DL Loss: 0.000000  | Accuracy: 98.1162%  ==


/data/wanh/CANAL/model.py:106: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:
AnnData object with n_obs × n_vars = 845 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'


cell type composition of this example bank:
(array(['acinar', 'alpha', 'beta', 'delta', 'ductal', 'gamma',
       'mesenchymal'], dtype=object), array([142, 142, 142, 142, 142,  76,  59]))


dataset composition from each stage of this example bank:
(array([1]), array([845]))




## stage 2

### load the pre-processed dataset ''Enge'' 

In [6]:
dataset_stage_2 = "Enge"
data_path_stage_2 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_2)
adata_stage_2 = sc.read_h5ad(data_path_stage_2)
cell_type_stage_2 = adata_stage_2.obs['cell_type1']
print(adata_stage_2)  
print(np.unique(np.array(cell_type_stage_2), return_counts=True))

AnnData object with n_obs × n_vars = 1782 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['acinar', 'alpha', 'beta', 'delta', 'ductal', 'mesenchymal'],
      dtype=object), array([332, 784, 267,  55, 300,  44]))


### fine-tune the CANAL model for the second stage

In [7]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_1, dataset = dataset_stage_2, 
            adata = adata_stage_2, cell_type = cell_type_stage_2, current_stage = 2, 
            is_final_stage = False, ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: AnnData object with n_obs × n_vars = 1782 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['acinar', 'alpha', 'beta', 'delta', 'ductal', 'mesenchymal'],
      dtype=object), array([332, 784, 267,  55, 300,  44]))
model constructing begin!

new cell types: []
example bank for experience replay: AnnData object with n_obs × n_vars = 845 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'rank', 'stage'
    var: 'highl

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


  ==  Begin finetuning: | Incrmental stage | Current stage: 2 | CANAL | Dataset: Pancreas Enge  ==
    ==  Epoch: 1 | Classification Loss: 0.732645 | Representation DL Loss: 0.094611  | Accuracy: 90.7619%  ==
    ==  Epoch: 1 | Validation CLS Loss: 2.961814 | Validation Representation DL Loss: 0.213464 | F1 Score: 0.926786  == | Current accuracy: 0.955598  ==
[[ 80   0   0   0   0   0   0]
 [  1 173   1   0   0   5   0]
 [  0   4  90   2   0   0   0]
 [  0   3   1  40   0   0   0]
 [  0   1   0   0  83   0   0]
 [  0   5   0   0   0  10   0]
 [  0   0   0   0   0   0  19]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.638844 | Representation DL Loss: 0.274230  | Accuracy: 94.7619%  ==
    ==  Epoch: 2 | Validation CLS Loss: 2.805388 | Validation Representation DL Loss: 0.231074 | F1 Score: 0.934943  == | Current accuracy: 0.967181  ==
[[ 78   0   0   0   0   0   0]
 [  0 195   0   0   0   1   0]
 [  0   1  76   4   0   0   0]
 [  0   1   0  33   0   0   0]
 [  0   2   0   

    ==  Epoch: 18 | Classification Loss: 0.524881 | Representation DL Loss: 0.189891  | Accuracy: 97.9524%  ==
    ==  Epoch: 18 | Validation CLS Loss: 3.263474 | Validation Representation DL Loss: 0.244640 | F1 Score: 0.962683  == | Current accuracy: 0.969112  ==
[[ 86   0   0   0   0   0   0]
 [  7 200   0   0   0   0   0]
 [  1   2  71   0   0   0   0]
 [  0   3   0  32   0   0   0]
 [  0   0   0   0  83   0   0]
 [  0   2   0   1   0  13   0]
 [  0   0   0   0   0   0  17]]
Patience: 5 / 10
    ==  Epoch: 19 | Classification Loss: 0.531891 | Representation DL Loss: 0.206761  | Accuracy: 97.5714%  ==
    ==  Epoch: 19 | Validation CLS Loss: 3.369882 | Validation Representation DL Loss: 0.187305 | F1 Score: 0.975497  == | Current accuracy: 0.974903  ==
[[ 82   0   0   0   0   0   0]
 [  2 188   1   0   0   0   0]
 [  1   2  83   1   0   0   0]
 [  0   6   0  33   0   0   0]
 [  0   0   0   0  81   0   0]
 [  0   0   0   0   0  17   0]
 [  0   0   0   0   0   0  21]]
Patience: 6 / 10


    ==  Epoch: 35 | Classification Loss: 0.484148 | Representation DL Loss: 0.180605  | Accuracy: 99.0476%  ==
    ==  Epoch: 35 | Validation CLS Loss: 3.298595 | Validation Representation DL Loss: 0.174113 | F1 Score: 0.982748  == | Current accuracy: 0.990347  ==
[[100   0   0   0   0   0   0]
 [  1 190   0   0   0   0   0]
 [  0   0  90   0   0   0   0]
 [  0   1   1  32   0   0   0]
 [  0   0   0   0  77   0   0]
 [  0   1   1   0   0  14   0]
 [  0   0   0   0   0   0  10]]
Patience: 6 / 10
    ==  Epoch: 36 | Classification Loss: 0.481255 | Representation DL Loss: 0.182213  | Accuracy: 99.3810%  ==
    ==  Epoch: 36 | Validation CLS Loss: 2.989938 | Validation Representation DL Loss: 0.176544 | F1 Score: 0.982944  == | Current accuracy: 0.982625  ==
[[ 96   0   0   0   0   0   0]
 [  2 174   3   0   0   0   0]
 [  0   0  76   0   0   0   0]
 [  0   4   0  29   0   0   0]
 [  0   0   0   0  94   0   0]
 [  0   0   0   0   0  16   0]
 [  0   0   0   0   0   0  24]]
Patience: 7 / 10


/data/wanh/CANAL/model.py:106: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:
AnnData object with n_obs × n_vars = 889 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'


cell type composition of this example bank:
(array(['acinar', 'alpha', 'beta', 'delta', 'ductal', 'gamma',
       'mesenchymal'], dtype=object), array([142, 142, 142, 142, 142,  76, 103]))


dataset composition from each stage of this example bank:
(array([1, 2]), array([506, 383]))




## stage 3

### load the pre-processed dataset ''Baron_human'' 

In [8]:
dataset_stage_3 = "Baron_human"
data_path_stage_3 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_3)
adata_stage_3 = sc.read_h5ad(data_path_stage_3)
cell_type_stage_3 = adata_stage_3.obs['cell_type1']
print(adata_stage_3)  
print(np.unique(np.array(cell_type_stage_3), return_counts=True))

AnnData object with n_obs × n_vars = 7951 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['acinar', 'activated_stellate', 'alpha', 'beta', 'delta', 'ductal',
       'endothelial', 'gamma', 'quiescent_stellate'], dtype=object), array([ 908,  268, 2176, 2367,  570, 1016,  241,  243,  162]))


### fine-tune the CANAL model for the third stage

In [9]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_2, dataset = dataset_stage_3, 
            adata = adata_stage_3, cell_type = cell_type_stage_3, current_stage = 3, is_final_stage = True,   
            ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: AnnData object with n_obs × n_vars = 7951 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['acinar', 'activated_stellate', 'alpha', 'beta', 'delta', 'ductal',
       'endothelial', 'gamma', 'quiescent_stellate'], dtype=object), array([ 908,  268, 2176, 2367,  570, 1016,  241,  243,  162]))
model constructing begin!

new cell types: ['activated_stellate', 'endothelial', 'quiescent_stellate']
example bank for experience replay: AnnData object with n_obs × n_vars = 889 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', 

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


    ==  Epoch: 1 | Classification Loss: 0.897652 | Representation DL Loss: 0.042492  | Accuracy: 87.1570%  ==
    ==  Epoch: 1 | Validation CLS Loss: 3.369426 | Validation Representation DL Loss: 0.082840 | F1 Score: 0.548260  == | Current accuracy: 0.848639  ==
[[187   0   0   0   0   0   2   0   0   0]
 [  2 439   6   0   0   5   0   0   0   0]
 [  4   5 489   7   0   0   0   0   0   0]
 [  0   0   6 114   0   0   0   0   0   0]
 [ 39   2   2   0 216   0   0   0   0   0]
 [  0  22   9   3   0  32   0   0   0   0]
 [  0   0   0   0   0   0  20   0   0   0]
 [  0   0   0   0   0   0  54   0   0   0]
 [  0  10  18   0  29   0   6   0   0   0]
 [  0   0   2   0   0   0  34   0   0   0]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.770181 | Representation DL Loss: 0.238887  | Accuracy: 89.9576%  ==
    ==  Epoch: 2 | Validation CLS Loss: 3.496991 | Validation Representation DL Loss: 0.277476 | F1 Score: 0.763874  == | Current accuracy: 0.920635  ==
[[212   0   0   0   1   0 

    ==  Epoch: 13 | Classification Loss: 0.548298 | Representation DL Loss: 0.211981  | Accuracy: 98.9250%  ==
    ==  Epoch: 13 | Validation CLS Loss: 3.758121 | Validation Representation DL Loss: 0.213382 | F1 Score: 0.970261  == | Current accuracy: 0.973923  ==
[[238   0   0   0   1   0   0   0   0   0]
 [  0 443   1   1   0   0   0   0   0   0]
 [  4   2 486   6   0   4   0   0   0   0]
 [  0   0   4 136   0   0   0   0   0   0]
 [  8   1   0   2 210   0   0   0   0   0]
 [  0   8   0   3   0  52   0   0   0   0]
 [  0   0   0   0   0   0  18   0   0   0]
 [  0   0   0   0   0   0   0  60   0   0]
 [  0   0   0   0   0   0   0   0  47   0]
 [  0   0   0   0   0   0   0   1   0  28]]
Patience: 2 / 10
    ==  Epoch: 14 | Classification Loss: 0.544526 | Representation DL Loss: 0.210421  | Accuracy: 99.0099%  ==
    ==  Epoch: 14 | Validation CLS Loss: 3.543711 | Validation Representation DL Loss: 0.203992 | F1 Score: 0.969811  == | Current accuracy: 0.974490  ==
[[204   0   0   0   1 

    ==  Epoch: 25 | Classification Loss: 0.540041 | Representation DL Loss: 0.235430  | Accuracy: 99.1655%  ==
    ==  Epoch: 25 | Validation CLS Loss: 3.968665 | Validation Representation DL Loss: 0.217216 | F1 Score: 0.969445  == | Current accuracy: 0.974490  ==
[[202   0   0   0   3   0   0   0   0   2]
 [  2 465   1   0   4   0   0   0   0   0]
 [  0   4 502   5   0   1   0   0   0   0]
 [  0   3   4 135   0   0   0   0   0   0]
 [  1   0   0   0 213   0   0   0   0   0]
 [  0  11   0   2   0  59   0   0   0   0]
 [  0   0   0   0   0   0  20   0   0   0]
 [  0   0   0   0   0   0   0  37   0   0]
 [  0   0   0   0   0   0   0   0  53   0]
 [  0   0   1   0   0   0   0   1   0  33]]
Patience: 1 / 10
    ==  Epoch: 26 | Classification Loss: 0.540302 | Representation DL Loss: 0.230620  | Accuracy: 99.1938%  ==
    ==  Epoch: 26 | Validation CLS Loss: 3.945568 | Validation Representation DL Loss: 0.224271 | F1 Score: 0.969770  == | Current accuracy: 0.974490  ==
[[197   0   0   0   2 

## load the unlabeled test data and apply CANAL to predict cell types

In [10]:
dataset_test = sc.read_h5ad("./data/{}/{}_test.h5ad".format(experiments, experiments))
print(dataset_test)
pred_cell_type = CANAL.predict(adata_predict = dataset_test, ckpt_dir = './ckpts/', experiments = experiments,
                               stage_num=3, dataset = dataset_stage_3)

AnnData object with n_obs × n_vars = 2000 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'batch2'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    ==  Begin predicting after 3 fine-tuning stages: | Experiments: Pancreas ==
Annotation: ['acinar' 'alpha' 'beta' 'delta' 'ductal' 'gamma' 'mesenchymal'
 'activated_stellate' 'endothelial' 'quiescent_stellate'] 



## evaluate the annotation performance

In [11]:
print(np.unique(np.array(dataset_test.obs['cell_type1']),return_counts=True))
true_celltype = np.array(dataset_test.obs['cell_type1']) 
CANAL.evaluation(pred_cell_type=pred_cell_type, true_celltype=true_celltype)

(array(['acinar', 'activated_stellate', 'alpha', 'beta', 'delta', 'ductal',
       'endothelial', 'gamma', 'mesenchymal', 'quiescent_stellate'],
      dtype=object), array([234,  16, 784, 415, 133, 295,  15,  67,  30,  11]))
Begin evaluation!

  ==  Predict total accuracy: 0.960500 ==|== F1 Score: 0.946200  ==|==  ARI: 0.923800 == 

Confusion matrix:
[[211   0   1   0   0  22   0   0   0   0]
 [  0  14   0   1   0   1   0   0   0   0]
 [  1   0 768   6   5   2   0   2   0   0]
 [  0   0   7 399   5   1   0   2   0   1]
 [  0   0   1   3 129   0   0   0   0   0]
 [  2   0   0   2   0 288   0   0   3   0]
 [  0   0   0   0   0   0  15   0   0   0]
 [  0   0   6   4   0   0   0  57   0   0]
 [  0   0   0   0   0   0   0   0  30   0]
 [  0   0   0   1   0   0   0   0   0  10]]
